In [1]:
# coding:utf-8
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',100)
import time

In [ ]:
# 整理财务数据 
# 生成包含财务信息的dataframe：df

In [116]:
bs = open('bs.txt','r',encoding='UTF-8').read()
income = open('is.txt','r',encoding='UTF-8').read()

In [120]:
def numeric(x):
    # 将str：19,000转化为 float：19000 
    try:
        return(float(x.replace(',','')))
    except:
        pass

def percent_to_num(x):
    # 将 str：+19% 转化为 float：0.19；将 str：-19% 转化为 float: -0.19
    try:
        if x[0] =='+':
            return(float(x.replace('+','').replace('%',''))/100)
        elif x[0]=='-':
            return(float(x.replace('-','').replace('%',''))/100)
        else:
            return(0)
    except:
        pass

In [74]:
# 爬取上市当天记账货币兑港币的汇率
def get_exchange_rate(driver,date,hold,target):
    ele_date = driver.find_element_by_xpath('//*[@id="end_date_input"]')
    ele_date.click()
    ele_date.send_keys(date)
    ele_date.send_keys(Keys.ENTER)
    ele_hold = driver.find_element_by_xpath('//*[@id="quote_currency_input"]')
    ele_hold.send_keys(hold)
    ele_hold.send_keys(Keys.ENTER)
    ele_target = driver.find_element_by_xpath('//*[@id="base_currency_input"]')
    ele_target.send_keys(target)
    ele_target.send_keys(Keys.ENTER)
    time.sleep(0.5)
    ele_avg = driver.find_element_by_xpath('//*[@id="bidAskBidAvg"]')
    rate = ele_avg.text
    return(rate)

In [34]:
# 生成dataframe ：df_bs
# 按\n分离
bs1 = bs.split('\n')
# 生成df
df_bs = pd.DataFrame()
for i in bs1[:-1]:
    row = i.split('|')[:-1]
    df_bs = df_bs.append([row])
# 加上header
df_bs.columns = ['code','date','asset1','cash1','lia1','asset2','cash2','lia2','asset3','cash3','lia3','asset4','cash4','lia4','asset5','cash5','lia5']
# 将string转化为float
df_bs[['asset1','cash1','lia1','asset2','cash2','lia2','asset3','cash3','lia3','asset4','cash4','lia4','asset5','cash5','lia5']]=df_bs[['asset1','cash1','lia1','asset2','cash2','lia2','asset3','cash3','lia3','asset4','cash4','lia4','asset5','cash5','lia5']].applymap(lambda x:numeric(x))

C:\Users\32983\Anaconda3\envs\python36\lib\site-packages\pandas\core\frame.py:5186: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  other = other.loc[:, self.columns]
C:\Users\32983\Anaconda3\envs\python36\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [55]:
# 生成dataframe: df_income
# 按\n分离
income1 = income.split('\n')
# 生成df
df_income = pd.DataFrame()
for i in income1[:-1]:
    row = [i.split('|')[-1]]+i.split('|')[:-1]
    df_income = df_income.append([row])
# 加上header
df_income.columns = ['currency','code','date','sale1','opt_pro1','holder_pro1','sale2','opt_pro2','holder_pro2','sale3','opt_pro3','holder_pro3','sale4','opt_pro4','holder_pro4','sale5','opt_pro5','holder_pro5']
# 将string转化为float
df_income[['sale1','opt_pro1','holder_pro1','sale2','opt_pro2','holder_pro2','sale3','opt_pro3','holder_pro3','sale4','opt_pro4','holder_pro4','sale5','opt_pro5','holder_pro5']]=df_income[['sale1','opt_pro1','holder_pro1','sale2','opt_pro2','holder_pro2','sale3','opt_pro3','holder_pro3','sale4','opt_pro4','holder_pro4','sale5','opt_pro5','holder_pro5']].applymap(lambda x:numeric(x))

C:\Users\32983\Anaconda3\envs\python36\lib\site-packages\pandas\core\frame.py:5186: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  other = other.loc[:, self.columns]
C:\Users\32983\Anaconda3\envs\python36\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [105]:
# 将bs、income合并
df = pd.merge(df_bs,df_income,how='left',on=['code','date'])
# 现在的date只有年、月，将其替换为年、月、日的date，为下一步查找汇率做准备
stock = open('stocks.txt','r').read().split('|')
df_stock = pd.DataFrame()
for i in stock[1:-1]:
    df_stock = df_stock.append([i.split(',')])
df_stock.columns=['date','code']

df.drop('date',axis=1,inplace=True)
df = pd.merge(df,df_stock,how ='left',on='code')

In [131]:
# 统一货币名称
df.replace('人民币','CNY',inplace=True) # onshore exchange rate
df.replace('新加坡元','SGD',inplace=True)
df.replace('欧罗','EUR',inplace=True)
df.replace('欧罗','EUR',inplace=True)
df.replace('港元','HKD',inplace=True)
df.replace('美元','USD',inplace=True)
df.replace('澳元','MOP',inplace=True)
df.replace('加元','CAD',inplace=True)
df.replace('日圆','JPY',inplace=True)


In [2]:
# 收集ipo当日的汇率
#df['rate'] = np.nan
#url_rate = 'https://www1.oanda.com/lang/cns/currency/converter/'
#driver = webdriver.Chrome()
#driver.get(url_rate)

for i in range(1026,len(df)):
    date = df.loc[i,'date']
    hold = df.loc[i,'currency']
    df.loc[i,'rate'] = get_exchange_rate(driver,date,hold,'HKD')

In [82]:
df= df.drop_duplicates()

In [87]:
#df.to_excel('financial_data_20190808_1.xlsx')

In [94]:
# 所有财务数据的单位为千，转化为元
# 再按照汇率转化为港元
header = ['sale1','opt_pro1','holder_pro1','sale2','opt_pro2','holder_pro2','sale3','opt_pro3','holder_pro3','sale4','opt_pro4','holder_pro4','sale5','opt_pro5','holder_pro5','asset1','cash1','lia1','asset2','cash2','lia2','asset3','cash3','lia3','asset4','cash4','lia4','asset5','cash5','lia5']
df[header] = df[header].applymap(lambda x:x*1000)
df[header] = np.multiply(df[header],df['rate'].reshape(len(df),1)) # (778,30) * (778,1) 的 Multiply arguments element-wise.

C:\Users\32983\Anaconda3\envs\python36\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [ ]:
# 整理ipo数据
#　生成包含ipo信息的dataframe：df6

In [ ]:
with open('ipo_info.txt') as f:
    d = f.readlines()

In [ ]:
df_ipo = pd.DataFrame()
for i in d[1:]:
    df_ipo = df_ipo.append([[j.strip() for j in i.split('\t')[1:-1]]]) # 第一项为空值，不要；最后一项为网址，不需要


In [ ]:
# df1 多了一列，因此将df分为df1\df2，处理之后再将二者concat
df1 = df_ipo[df_ipo.iloc[:,30]=='N/A'] # long
df2 = df_ipo[df_ipo.iloc[:,30] != 'N/A'] # short

df3 = df1.drop([27,28,29,30,31],axis=1)
df4 = df2.drop([27,28,29,30],axis=1)

In [ ]:
# 加上列名，再根据列名concat
c = ['date', 'code', 'name', 'category', 'ipo_price', 'buy_ratio', 'one_hand', 'draw_prob', 'firstday_performance', 'now_price', 'total_performance', 'firm', 'fund', 'individual', 'period', 'pricingDate', 'publicDate', 'backDate', 'ipoDate', 'ipoArea', 'industry1', 'backgroud', 'marketArea', 'hand', 'priceGap', 'marketValue', 'hkShare', 'sponsor1','temp1', 'sponsor2','temp2', 'sponsor3','temp3', 'sponsor4','temp4', 'sponsor5','temp5', 'sponsor6']
c1 = c+['temp6']
df3.columns=c
df4.columns=c1
# drop冗余列
df3.drop(['temp1','temp2','temp3','temp4','temp5','sponsor6'],axis=1,inplace=True)
df4.drop(['temp1','temp2','temp3','temp4','temp5','temp6','sponsor6'],axis=1,inplace=True)

In [ ]:
#　df5为结构化后raw data
df5 = pd.concat([df3,df4])
df5.index=range(len(df5))
# 在df5的基础上进行数据清洗
df6 = df5.replace('N/A',np.nan)


In [ ]:
def low(x):
    try:
        return(x.split('-')[0])
    except:
        pass
def high(x):
    try:
        return(x.split('-')[-1])
    except:
        pass

def highM(x):
    try:
        return(x.split('-')[-1].replace(',',''))
    except:
        pass
def lowM(x):
    try:
        return(x.split('-')[0].replace(',',''))
    except:
        pass
def percent_to_float(x):
    try:
        return(float(x.replace('%',''))/100)
    except:
        pass
#　构造年份、月份变量
def yearAndMonth(x):
    try:
        return(x.split('/')[0]+'-'+x.split('/')[1])
    except:
        pass

In [ ]:
# 提取最高价和最低价
# firm\fund\individual的单位为百万，需转化为元
df6['low'] = np.nan

df6['low']= df6['priceGap'].apply(lambda x: low(x))
df6['high'] = df6['priceGap'].apply(lambda x:high(x))
df6['lowMarketValue'] = df6['marketValue'].apply(lowM)
df6['highMarketValue'] = df6['marketValue'].apply(highM)
df6['firstDay'] = df6['firstday_performance'].apply(percent_to_float)
df6.replace(['認購不足','认购不足',None],np.nan,inplace=True)
df6.drop(df6[df6['firm']=='date'].index,axis=0,inplace=True)
df6[['ipo_price','buy_ratio','firm','fund','individual','hand','low','high','lowMarketValue','highMarketValue']] = df6[['ipo_price','buy_ratio','firm','fund','individual','hand','low','high','lowMarketValue','highMarketValue']].applymap(lambda x : float(x))
df6['firmRatio'] = df6['firm']*2*1000000/(df6['lowMarketValue']+df6['highMarketValue'])
df6['fundRatio'] = df6['fund']*2*1000000/(df6['lowMarketValue']+df6['highMarketValue'])
df6['indiRatio'] = df6['individual']*2*1000000/(df6['lowMarketValue']+df6['highMarketValue'])
df6['percent'] = df6['draw_prob'].apply(lambda x: percent_to_float(x))
# drop 异常值
df6.drop(288,axis=0,inplace=True)
df6['yearNmonth'] = df6['date'].apply(lambda x:yearAndMonth(x))

In [ ]:
# 清洗数据: 繁体转化为简体，保荐人英文名转化为中文名，确保每类值只有唯一名称
df6['category'].replace({'乳製品':'乳制品','仓储物流':'仓储物流','休閒娛樂':'休闲娱乐','供電供熱':'供电供热','保險': '保险','倉儲物流':'仓储物流',\
                            '公用運輸':'公用运输','公路及鐵路':'公路及铁路','其他支援服務':'其他支援服务','其他礦物':'其他矿物','包裝食品':'包装食品',\
                            '化工產品':'化工产品','化肥農藥':'化肥农药','半導體':'半导体','印刷及包裝':'印刷及包装','地產投資':'地产投资','地產發展':'地产发展',\
                            '家用電器':'家用电器','工業用品':'工业用品','廣告':'广告','建築及裝修':'建筑及装修','建築材料':'建筑材料','影視娛樂':'影视娱乐',\
                            '投資及資產管理':'投资及资产管理','旅遊及觀光':'旅游及观光','服裝':'服装','服飾配件及皮革':'服饰配件及皮革','林木業':'林木业',\
                            '機械裝備及器材':'机械装备及器材','殯儀':'殡仪','水務供應':'水务供应','汽車及零部件':'汽车及零部件','汽車零售及維護維修':'汽车零售及维护维修',\
                            '煙酒':'烟酒','燃氣供應':'燃气供应','物業管理及代理':'物业管理及代理','環保':'环保','石油及天然氣':'石油及天然气','系統開發及信息科技顧問':\
                            '系统开发及信息科技顾问','紡織及制衣':'纺织及制衣','美容及個人護理':'美容及个人护理','航空服務':'航空服务','航運及港務':'航运及港务',\
                            '藥品':'药品','證券':'证券','資訊科技器材':'资讯科技器材','超市/百貨/商城':'超市/百货/商城','軟件開發':'软件开发','農畜漁產品':'农畜渔产品',\
                            '醫療器械及服務':'医疗器械及服务','銀行':'银行','鋼鐵':'钢铁','鐘錶珠寶':'钟表珠宝','零售/貿易':'零售/贸易','電子元器件':'电子元器件',\
                            '電子商貿及互聯網服務':'电子商贸及互联网服务','電子消費品':'电子消费品','電氣設備':'电气设备','電視廣播':'电视广播','電訊及網絡器材':'电讯及网络设备',\
                            '電訊服務':'电讯服务','鞋類':'鞋类','食品添加劑及調味料':'食品添加剂及调味料','餐飲':'餐饮','綜合企業':'综合企业'},inplace=True)
df6['ipoArea'].replace({'香港联合交易所有限公司':'主板','香港联合交易所有限公司 (主板)':'主板','香港联合交易所有限公司(主板)':'主板',\
                       '香港联合交易所有限公司（主板）':'主板','香港联合交易所有限公司 (创业板)':'创业板','香港联合交易所有限公司(创业板)':'创业板',\
                       '香港联合交易所有限公司（创业板）':'创业板','香港聯合交易所有限公司(创业板)':'创业板'},inplace=True)
for i in ['sponsor1','sponsor2','sponsor3','sponsor4']:
    df6[i].replace({'Merrill Lynch Far East Limited':'美林远东有限公司','Credit Suisse (Hong Kong) Limited':'瑞士信贷(香港)有限公司','J.P. Morgan Securities (Far East) Limited':'摩根大通证券(远东)有限公司',\
                           'UBS Securities Hong Kong Limited':'瑞银证券香港有限公司'},inplace=True)

In [ ]:
# 增加特征:保荐人数
df6['sponsorNum'] = 5-pd.isnull(df6[['sponsor1','sponsor2','sponsor3','sponsor4','sponsor5']]).sum(axis=1)


In [98]:
# 将财务数据与ipo数据合并，并新加特征
df7 = pd.merge(df,df6,on=['code'])


In [113]:
# 新增特征
df7['share'] = df7['lowMarketValue']/df7['low']
df7['pe'] = df7['ipo_price']/(df7['holder_pro1']/df7['share'])
df7['ps'] =df7['ipo_price']/(df7['sale1']/df7['share'])
df7['pb'] =df7['ipo_price']/((df7['asset1']-df7['lia1'])/df7['share'])
df7['ba1'] = df7['lia1']/df7['asset1']
df7['ca1'] = df7['cash1']/df7['asset1']
df7['proSale1'] = df7['opt_pro1']/df7['sale1']
df7['holderSale1'] = df7['holder_pro1']/df7['sale1']

df7['assGro1'] = (df7['asset1']-df7['asset2'])/df7['asset2']
df7['assGro2'] = (df7['asset2']-df7['asset3'])/df7['asset3']
df7['assGro3'] = (df7['asset3']-df7['asset4'])/df7['asset4']
df7['assGro4'] = (df7['asset4']-df7['asset5'])/df7['asset5']

df7['saleGro1'] = (df7['sale1']-df7['sale2'])/df7['sale2']
df7['saleGro2'] = (df7['sale2']-df7['sale3'])/df7['sale3']
df7['saleGro3'] = (df7['sale3']-df7['sale4'])/df7['sale4']
df7['saleGro4'] = (df7['sale4']-df7['sale5'])/df7['sale5']

df7['opt_proGro1'] = (df7['opt_pro1']-df7['opt_pro2'])/df7['opt_pro2']
df7['opt_proGro2'] = (df7['opt_pro2']-df7['opt_pro3'])/df7['opt_pro3']
df7['opt_proGro3'] = (df7['opt_pro3']-df7['opt_pro4'])/df7['opt_pro4']
df7['opt_proGro4'] = (df7['opt_pro4']-df7['opt_pro5'])/df7['opt_pro5']

In [131]:
df7.replace(np.inf,np.nan,inplace=True)
df7.replace(-np.inf,np.nan,inplace=True)

In [133]:
df7.describe()

,code,ipo_price,buy_ratio,one_hand,firstday_performance,now_price,firm,fund,individual,hand,low,high,lowMarketValue,highMarketValue,firstDay,firmRatio,fundRatio,indiRatio,year,month,percent,sponsorNum,asset1,cash1,lia1,asset2,cash2,lia2,asset3,cash3,lia3,asset4,cash4,lia4,asset5,cash5,lia5,sale1,opt_pro1,holder_pro1,sale2,opt_pro2,holder_pro2,sale3,opt_pro3,holder_pro3,sale4,opt_pro4,holder_pro4,sale5,opt_pro5,holder_pro5,rate,share,pe,ps,pb,ba1,ca1,proSale1,holderSale1,assGro1,assGro2,assGro3,assGro4,saleGro1,saleGro2,saleGro3,saleGro4,opt_proGro1,opt_proGro2,opt_proGro3,opt_proGro4
count,770.000000,705.000000,548.000000,616.000000,747.000000,765.000000,770.000000,770.000000,770.000000,769.000000,705.000000,705.000000,6.550000e+02,6.550000e+02,747.000000,655.000000,655.000000,655.000000,770.000000,770.000000,617.000000,770.000000,7.660000e+02,7.660000e+02,7.660000e+02,6.250000e+02,6.250000e+02,6.250000e+02,4.180000e+02,4.180000e+02,4.180000e+02,1.140000e+02,1.140000e+02,1.140000e+02,6.000000e+00,6.000000e+00,6.000000e+00,7.660000e+02,7.660000e+02,7.660000e+02,6.250000e+02,6.250000e+02,6.250000e+02,4.180000e+02,4.180000e+02,4.180000e+02,1.140000e+02,1.140000e+02,1.140000e+02,6.000000e+00,6.000000e+00,6.000000e+00,766.000000,6.550000e+02,645.000000,648.000000,651.000000,7.660000e+02,766.000000,763.000000,763.000000,625.000000,418.000000,114.000000,6.000000,620.000000,415.000000,114.000000,6.000000,624.000000,415.000000,113.000000,6.000000
mean,4344.707792,3.553807,138.812774,32.228896,0.516605,3.572945,315.949682,80.002650,10.241500,4392.132640,3.147539,3.799722,5.049425e+09,6.131533e+09,0.444776,0.042464,0.007420,0.003461,2016.831169,6.745455,0.718091,1.201299,1.672484e+10,3.698073e+09,3.475553e+09,1.167057e+10,2.857020e+09,2.404135e+09,7.376764e+09,1.740635e+09,2.106278e+09,4.555965e+09,9.463065e+08,7.988723e+08,4.333344e+08,5.023938e+07,1.898861e+08,3.271509e+09,4.037792e+08,1.589148e+08,2.129876e+09,2.305116e+08,1.368345e+08,1.379225e+09,1.441465e+08,5.310130e+07,1.085260e+09,8.327988e+07,4.110092e+07,9.475823e+07,1.324936e+07,1.093530e+07,1.594026,1.466449e+09,404.165469,31.911525,13.821602,1.642652e-01,0.263354,-1.031940,-1.040414,0.298659,0.420929,0.428769,1.121327,0.648181,0.739000,0.421950,0.694954,-0.144381,0.930010,-0.188872,0.603397
std,3027.149156,8.619854,405.635535,102.318511,1.431210,8.382229,1264.755673,541.321159,153.485112,4149.428911,7.474439,8.678223,2.336902e+10,2.894421e+10,1.455167,0.110607,0.032810,0.022802,1.435230,3.690586,0.328367,0.715606,8.215229e+10,2.302588e+10,2.230254e+10,5.944577e+10,1.905997e+10,1.822436e+10,3.595869e+10,1.098922e+10,1.740689e+10,2.023835e+10,6.390030e+09,3.646772e+09,8.405871e+08,7.682391e+07,3.932988e+08,1.869300e+10,1.734454e+09,2.357286e+09,1.173476e+10,1.330144e+09,9.664321e+08,5.325079e+09,8.750709e+08,9.295405e+08,3.190720e+09,2.523455e+08,2.299174e+08,5.204150e+07,4.869778e+06,4.032393e+06,1.636169,6.966153e+09,4986.480703,324.776661,161.715898,2.040162e-01,0.195784,28.209703,28.170836,0.496626,1.361844,0.874392,1.146188,9.520219,7.416639,1.383269,0.767367,6.060014,13.767544,8.834717,0.915864
min,1.000000,0.130000,0.000000,1.000000,0.000000,0.030000,0.000000,0.000000,0.000000,100.000000,0.100000,0.140000,1.000000e+08,1.200000e+08,-0.658300,0.000000,0.000000,0.000000,2014.000000,1.000000,0.008000,0.000000,1.158544e+06,9.994796e+04,0.000000e+00,1.578064e+06,8.987840e+03,0.000000e+00,1.550912e+06,0.000000e+00,0.000000e+00,7.640747e+06,1.495140e+05,0.000000e+00,2.938667e+07,5.222000e+06,0.000000e+00,0.000000e+00,-4.378809e+09,-5.176641e+10,0.000000e+00,-7.159070e+09,-6.626309e+09,0.000000e+00,-9.698080e+09,-1.203896e+10,1.280578e+06,-1.513305e+09,-1.509342e+09,2.587610e+07,1.081700e+07,7.551152e+06,0.064650,1.729667e+07,-1412.429379,0.034475,-12.281379,0.000000e+00,0.001729,-768.429336,-767.038544,-0.692098,-0.457742,-0.273924,0.078906,-0.871411,-0.694437,-0.268829,0.049774,-141.930389,-78.346067,-90.394025,-0.199508
25%,1712.250000,0.500000,3.800000,1.000000,0.026100,0.3

In [ ]:
df7.to_excel('ipo_20190808.xlsx')